In [20]:
payload = {
    "datasetName": "random_10",
    "gaParameters": {
        "maxNumberOfIter":300,
        "populationSize": 100,
        "maximumWildCardCount": 0,
        "orientationWeights": [
            1,
            1,
            0.5
        ],
        "geneticAlgorithm": "simpleGa",
        "mate": "normalizedProbabilityVectorSum",
        "mutate": "flipOnePartAtRandom",
        "select": "tournament",
        "objective": "simple",
        "evaluator": "ga",
        "placingHeuristics": "corner",
        "populationDivisionCounts": {
            "elite": 0.2,
            "average": 0.6,
            "worst": 0.2,
            "children": 0.3,
            "mutant": 0.2,
            "winner": 0.2,
            "random": 0.1
        },
        "initialPopulationDivisionCounts": {
            "random": 0.7,
            "greedy": 0.3
        }
    },
    "objectiveParameters": {
        "name": "simple",
        "params": {
            "overlappingPenalizationConstant": 1000,
            "outsideOfAllocatedAreaPenalizationConstant": 0
        }
    }
}

url = "http://localhost:9000/compute/dataset"

headers = {
  'Content-Type': 'application/json'
}

In [6]:
import os
import time
import requests
import json
from pathlib import Path
import os
import shutil
from datetime import datetime
import math


def createDirectory(directory, removeAll=False):
    '''
    Creates a directory if it doesn't exist and removes all files
    from the directory if it already exists and removeAll is True.
    '''
    if os.path.exists(directory) and removeAll:
        shutil.rmtree(directory) # Remove all files in the directory
        print(f'All files in {directory} have been removed.')

    if not os.path.exists(directory):
        os.makedirs(directory) # Create the directory

def checkforFileBlocking(outDir):
    start_time = time.time()  # Get the current time in seconds
    while True:
        if os.path.exists(os.path.join(outDir, "computation_result.json")):
            elapsed_time = time.time() - start_time  # Calculate the elapsed time in seconds
            print(f"{elapsed_time:.2f} sec")
            break
        else:
            time.sleep(1)  # Pause for 5 seconds before checking again

## Population size x best solution

In [10]:
OUT_DIR_BASE = 'out/cluster_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

for i in range(10):
    OUT_DIR = f'{OUT_DIR_BASE}/{i+1}'
    print(f'====== RUN {i+1}/{10} ======')
    createDirectory(OUT_DIR)
    for dataset in ['cluster_3_10', 'cluster_4_20']:
        with open(f'/home/slarty/Documents/master-thesis/src/impl/public/datasets/cluster/{dataset}.json', 'r') as file:
            layout = json.load(file)['data']['layout']
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
        payload['datasetName'] = dataset
        size = int(dataset.split('_')[1])
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 1000
        r = requests.request("POST", url, headers=headers, data=json.dumps(payload))
        src = r.json()['outputDirectory'] 
        checkforFileBlocking(src)
        dst = OUT_DIR + '/' + src.split('/')[-1]
        shutil.copytree(src, dst)

print('\nComputation done')

====== RUN 1/10 ======
851.86 sec
5878.29 sec
====== RUN 2/10 ======
851.85 sec
5859.40 sec
====== RUN 3/10 ======
851.86 sec
5848.45 sec
====== RUN 4/10 ======
852.87 sec
5871.55 sec
====== RUN 5/10 ======
855.87 sec


KeyboardInterrupt: 

# Cluster

In [10]:
OUT_DIR_BASE = 'out/cluster_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

for i in range(10):
    OUT_DIR = f'{OUT_DIR_BASE}/{i+1}'
    print(f'====== RUN {i+1}/{10} ======')
    createDirectory(OUT_DIR)
    for dataset in ['cluster_3_10', 'cluster_4_20']:
        with open(f'/home/slarty/Documents/master-thesis/src/impl/public/datasets/cluster/{dataset}.json', 'r') as file:
            layout = json.load(file)['data']['layout']
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
        payload['datasetName'] = dataset
        size = int(dataset.split('_')[1])
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 1000
        r = requests.request("POST", url, headers=headers, data=json.dumps(payload))
        src = r.json()['outputDirectory'] 
        checkforFileBlocking(src)
        dst = OUT_DIR + '/' + src.split('/')[-1]
        shutil.copytree(src, dst)

print('\nComputation done')

====== RUN 1/10 ======
851.86 sec
5878.29 sec
====== RUN 2/10 ======
851.85 sec
5859.40 sec
====== RUN 3/10 ======
851.86 sec
5848.45 sec
====== RUN 4/10 ======
852.87 sec
5871.55 sec
====== RUN 5/10 ======
855.87 sec


KeyboardInterrupt: 

# Packing

In [13]:
OUT_DIR_BASE = 'out/packing_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

for i in range(1):
    OUT_DIR = f'{OUT_DIR_BASE}/{i+1}'
    print(f'====== RUN {i+1}/{10} ======')
    createDirectory(OUT_DIR)
    for dataset in ['packing_10', 'packing_20']:
        with open(f'/home/slarty/Documents/master-thesis/src/impl/public/datasets/packing/{dataset}.json', 'r') as file:
            layout = json.load(file)['data']['layout']
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
        payload['datasetName'] = dataset
        size = int(dataset.split('_')[1])
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 1000
        r = requests.request("POST", url, headers=headers, data=json.dumps(payload))
        src = r.json()['outputDirectory'] 
        checkforFileBlocking(src)
        dst = OUT_DIR + '/' + src.split('/')[-1]
        shutil.copytree(src, dst)

print('\nComputation done')

====== RUN 1/10 ======
637.66 sec
3208.31 sec

Computation done


# Biased sparse cluster

In [27]:
OUT_DIR_BASE = 'out/biased_sparse_cluster_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

for i in range(1):
    OUT_DIR = f'{OUT_DIR_BASE}/{i+1}'
    print(f'====== RUN {i+1}/{10} ======')
    createDirectory(OUT_DIR)
    for dataset in ['biased_sparse_cluster_2_10']:
        with open(f'/home/slarty/Documents/master-thesis/src/impl/public/datasets/biased_sparse_cluster/{dataset}.json', 'r') as file:
            layout = json.load(file)['data']['layout']
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
        payload['datasetName'] = dataset
        size = 30
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 1000
        r = requests.request("POST", url, headers=headers, data=json.dumps(payload))
        print(r.json())
        src = r.json()['outputDirectory'] 
        checkforFileBlocking(src)
        dst = OUT_DIR + '/' + src.split('/')[-1]
        shutil.copytree(src, dst)

print('\nComputation done')

====== RUN 1/10 ======
{'id': 'biased_sparse_cluster_2_10_EC86D', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/001_biased_sparse_cluster_2_10_EC86D'}


KeyboardInterrupt: 

# Maximum wild card count vs computation time

In [18]:
OUT_DIR_BASE = 'out/wild_card_comp_time_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

for i in range(2, 6):
    OUT_DIR = f'{OUT_DIR_BASE}/{i+1}'
    print(f'====== RUN {i+1}/{10} ======')
    createDirectory(OUT_DIR)
    for dataset in ['random_10']:
        with open(f'/home/slarty/Documents/master-thesis/src/impl/public/datasets/random/{dataset}.json', 'r') as file:
            layout = json.load(file)['data']['layout']
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = 100
        payload['gaParameters']['maxNumberOfIter'] = 300
        
        for maximumWildCardCount in range(0,10):
            print(maximumWildCardCount)
            payload['gaParameters']['maximumWildCardCount'] = maximumWildCardCount
            r = requests.request("POST", url, headers=headers, data=json.dumps(payload))
            src = r.json()['outputDirectory'] 
            checkforFileBlocking(src)
            dst = OUT_DIR + '/' + src.split('/')[-1]
            shutil.copytree(src, dst)

print('\nComputation done')

====== RUN 3/10 ======
0
22.02 sec
1
37.04 sec
2
68.07 sec
3
109.11 sec
4
167.17 sec
5
218.23 sec
6
268.27 sec
7
338.34 sec
8
376.38 sec
9
383.38 sec
====== RUN 4/10 ======
0
21.02 sec
1
37.04 sec
2
65.07 sec
3
113.11 sec
4
172.18 sec
5
225.23 sec
6
282.29 sec
7
342.34 sec
8
380.38 sec
9
374.37 sec
====== RUN 5/10 ======
0
25.03 sec
1
40.04 sec
2
69.07 sec
3
109.11 sec
4
159.16 sec
5
217.22 sec
6
268.27 sec
7
336.34 sec
8
373.37 sec
9
382.38 sec
====== RUN 6/10 ======
0
21.02 sec
1
37.04 sec
2
66.07 sec
3
108.11 sec
4
161.16 sec
5
212.21 sec
6
276.28 sec
7
341.34 sec
8
374.37 sec
9
373.37 sec

Computation done
